In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import openpyxl
import pycountry

In [2]:
lt_region_recode = pd.read_csv(r'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/Country Match/Recode dictionary.csv').drop(columns=['var','varnm'])[0:228]

lt_region_recode.loc[:,'varvaldesc'] = lt_region_recode.loc[:,'varvaldesc'].replace({'Bolivia (Plurinational State of)': 'Bolivia, Plurinational State of', 
                                    'United States Virgin Islands': 'Virgin Islands, U.S.',
                                    'Reunion': 'Réunion',
                                    'Micronesia (Fed. States of)':'Micronesia, Federated States of',
                                    'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines',
                                    'Curacao':'Curaçao',
                                    "Lao People's Dem. Republic" :"Lao People's Democratic Republic",
                                    "Dem. People's Rep. of Korea":"Korea, Democratic People's Republic of",
                                    "China, Macao SAR":"Macao",
                                    "China, Hong Kong SAR":"Hong Kong",
                                    "Cote d'Ivoire":"Côte d'Ivoire",
                                    'Venezuela (Bolivarian Republic of)':'Venezuela, Bolivarian Republic of',
                                    'Republic of Korea':'Korea, Republic of',
                                    'Iran (Islamic Republic of)':'Iran, Islamic Republic of',
                                    'State of Palestine':'Palestine, State of',
                                    'Turkey': 'Türkiye',
                                    'China, Taiwan Province of China':'Taiwan, Province of China',
                                    'Dem. Republic of the Congo':'Congo, The Democratic Republic of the',
                                    'Micronesia':'Micronesia, Federated States of',
                                    'Polynesia':'French Polynesia'
                                    
                                    })

def country_to_iso(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

lt_region_recode.loc[:,'iso_code'] = lt_region_recode['varvaldesc'].apply(lambda x: country_to_iso(x))

lt_region_recode.loc[:,'iso_code'] = lt_region_recode.loc[:,'iso_code'].fillna(lt_region_recode.loc[:,'varvaldesc'])

lt_region_recode

,varval,varvaldesc,iso_code
0,reg108,Burundi,BDI
1,reg174,Comoros,COM
2,reg262,Djibouti,DJI
3,reg232,Eritrea,ERI
4,reg231,Ethiopia,ETH
...,...,...,...
223,reg935,Asia,Asia
224,reg904,Latin America and the Caribbean,Latin America and the Caribbean
225,reg909,Oceania,Oceania
226,reg908,Europe,Europe


In [3]:
pop = pd.read_csv(f'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/SSP datasets/Population/pop.csv')

pop

,SSPs,Year,sex,Population,iso_code,age
0,SSP2,2020,m,3231.9,AFG,0
1,SSP2,2025,m,3527.3,AFG,0
2,SSP2,2030,m,3773.0,AFG,0
3,SSP2,2035,m,3943.3,AFG,0
4,SSP2,2040,m,4075.3,AFG,0
...,...,...,...,...,...,...
717565,SSP5,2080,f,8.3,ZWE,100
717566,SSP5,2085,f,15.4,ZWE,100
717567,SSP5,2090,f,22.0,ZWE,100
717568,SSP5,2095,f,31.3,ZWE,100


# premature deaths - adjusted results - with PM2.5 feedbacks effect 

In [37]:
diseases = ['COP', 'IHD', 'LRI', 'LUN', 'STR', 'TDM']

scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370']

def process_disease(scenario, disease, scenario_mortality_change):
    
    if scenario == 'ssp119' or scenario == 'ssp126':
        mortality_data_path = f'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/SSP datasets/Life Tables/Mortality Rates_Diseases Share/SSP1_mortalityrate_{disease}.csv'
    elif scenario == 'ssp245':
        mortality_data_path = f'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/SSP datasets/Life Tables/Mortality Rates_Diseases Share/SSP2_mortalityrate_{disease}.csv'
    elif scenario == 'ssp370':
        mortality_data_path = f'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/SSP datasets/Life Tables/Mortality Rates_Diseases Share/SSP3_mortalityrate_{disease}.csv'

    mortality_df = pd.read_csv(mortality_data_path)
    
    rr_data_path = f'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/GBD 2021/PM25_RR_AF_reference_ssp245/PM25_RR_AF_reference_ssp245_{scenario}_{disease}.csv'
    
    rr_df = pd.read_csv(rr_data_path)

    rr_df = rr_df[['iso_code', 'Year', 'RR_AVG_reference', 'RR_LOW_reference','RR_HIGH_reference','AF_AVG', 'AF_LOW', 'AF_HIGH']]
    
    merged_df = mortality_df.merge(rr_df, on=['iso_code', 'Year'], how='outer')

    if disease == 'LRI':
        pass
    else:
        merged_df.loc[merged_df['age'] < 25, merged_df.columns.str.contains('RR_')] = 1
        merged_df.loc[merged_df['age'] < 25, merged_df.columns.str.contains('AF_')] = 0

    
    merged_df.drop(columns=['cause'], inplace=True)
    
    merged_df.rename(columns={
        'RR_AVG_reference': f'{disease}_RR_AVG_reference',
        'RR_LOW_reference': f'{disease}_RR_LOW_reference',
        'RR_HIGH_reference': f'{disease}_RR_HIGH_reference',
        'AF_AVG': f'{disease}_AF_AVG',
        'AF_LOW': f'{disease}_AF_LOW', 
        'AF_HIGH': f'{disease}_AF_HIGH'

    }, inplace=True)
    
    merged_df.dropna(subset=['SSPs', f'{disease}_RR_AVG_reference'], axis=0, inplace=True)

    if scenario_mortality_change is None:
        scenario_mortality_change = merged_df
    else:
        common_columns = ['SSPs', 'Year', 'sex', 'iso_code', 'age', 'mx']
        scenario_mortality_change = scenario_mortality_change.merge(merged_df, on=common_columns, how='outer')

    return scenario_mortality_change

all_scenarios_df = []

for scenario in scenarios:

    scenario_mortality_change = None

    for disease in diseases:
        scenario_mortality_change = process_disease(scenario, disease, scenario_mortality_change)
        if scenario_mortality_change is None:
            raise ValueError(f"Processing disease {disease} resulted in None.")
    
    scenario_mortality_new_df = scenario_mortality_change.merge(lt_region_recode, on=['iso_code'], how='left')

    scenario_mortality_new_df['Scenario'] = scenario
    
    all_scenarios_df.append(scenario_mortality_new_df)

yo_AF_reference_RR2021 = pd.concat(all_scenarios_df, ignore_index=True)

adjusted_cumu_deaths = yo_AF_reference_RR2021.merge(pop,on=['SSPs', 'Year', 'sex', 'iso_code', 'age'],how='left')

adjusted_cumu_deaths = adjusted_cumu_deaths.dropna(subset=['Population','LRI_RR_AVG_reference'])

for disease in diseases:
    for i in ['AVG', 'LOW', 'HIGH']:
        adjusted_cumu_deaths[f'{disease}_delta_mort_{i}'] = adjusted_cumu_deaths['mx'] * adjusted_cumu_deaths[f'{disease}_AVG'] *adjusted_cumu_deaths[f'{disease}_AF_{i}']* adjusted_cumu_deaths[f'{disease}_RR_{i}_reference'] * adjusted_cumu_deaths[f'Population']


for i in ['AVG', 'LOW', 'HIGH']:
    adjusted_cumu_deaths[f'total_delta_mort_{i}'] = adjusted_cumu_deaths[[f'{disease}_delta_mort_{i}' for disease in diseases]].sum(axis=1)

adjusted_cumu_deaths

output_path = '/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/Health outcomes/Premature Deaths/adjusted_cumu_deaths.csv'
adjusted_cumu_deaths.to_csv(output_path, index=False)

adjusted_cumu_deaths

,SSPs,Year,sex,iso_code,age,mx,COP_AVG,COP_LOW,COP_HIGH,COP_RR_AVG_reference,...,LUN_delta_mort_HIGH,STR_delta_mort_AVG,STR_delta_mort_LOW,STR_delta_mort_HIGH,TDM_delta_mort_AVG,TDM_delta_mort_LOW,TDM_delta_mort_HIGH,total_delta_mort_AVG,total_delta_mort_LOW,total_delta_mort_HIGH
0,SSP1,2020.0,f,AFG,0.0,0.011760,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.594675,0.056406,1.025721
1,SSP1,2020.0,f,AFG,5.0,0.001139,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033347,0.003163,0.057519
2,SSP1,2020.0,f,AFG,10.0,0.000867,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019188,0.001820,0.033096
3,SSP1,2020.0,f,AFG,15.0,0.001353,0.006470,0.003094,0.011349,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012825,0.001216,0.022121
4,SSP1,2020.0,f,AFG,20.0,0.001860,0.005892,0.002868,0.009952,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013615,0.001291,0.023483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222383,SSP3,2050.0,m,ZWE,80.0,0.131022,0.041236,0.025676,0.061785,1.018463,...,0.005904,0.025564,0.017389,0.034606,0.015313,0.008263,0.023346,0.136708,0.063574,0.209896
222384,SSP3,2050.0,m,ZWE,85.0,0.192507,0.037578,0.024011,0.057540,1.018463,...,0.001933,0.014370,0.009774,0.019452,0.007691,0.004150,0.011726,0.076295,0.034056,0.118237
222385,SSP3,2050.0,m,ZWE,90.0,0.273660,0.025427,0.014721,0.037348,1.018463,...,0.000427,0.005531,0.003762,0.007487,0.002556,0.001379,0.003897,0.042047,0.013706,0.069230
222386,SSP3,2050.0,m,ZWE,95.0,0.363767,0.013584,0.007471,0.021601,1.018463,...,0.000044,0.001055,0.000718,0.001429,0.000370,0.000200,0.000564,0.015561,0.003171,0.027154


# premature deaths - benchmark results - without PM2.5 feedbacks effect 

In [41]:
benchmark_cumu_deaths = yo_AF_reference_RR2021.merge(pop,on=['SSPs', 'Year', 'sex', 'iso_code', 'age'],how='left')

benchmark_cumu_deaths = benchmark_cumu_deaths.dropna(subset=['Population','LRI_RR_AVG_reference'])

for disease in diseases:
    for i in ['AVG', 'LOW', 'HIGH']:
        benchmark_cumu_deaths[f'{disease}_delta_mort_{i}'] = benchmark_cumu_deaths['mx'] * benchmark_cumu_deaths[f'{disease}_AVG'] *benchmark_cumu_deaths[f'{disease}_AF_{i}'] * benchmark_cumu_deaths[f'Population']


for i in ['AVG', 'LOW', 'HIGH']:
    benchmark_cumu_deaths[f'total_delta_mort_{i}'] = benchmark_cumu_deaths[[f'{disease}_delta_mort_{i}' for disease in diseases]].sum(axis=1)

benchmark_cumu_deaths

output_path = '/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/Health outcomes/Premature Deaths/benchmark_cumu_deaths.csv'
benchmark_cumu_deaths.to_csv(output_path, index=False)

benchmark_cumu_deaths


,SSPs,Year,sex,iso_code,age,mx,COP_AVG,COP_LOW,COP_HIGH,COP_RR_AVG_reference,...,LUN_delta_mort_HIGH,STR_delta_mort_AVG,STR_delta_mort_LOW,STR_delta_mort_HIGH,TDM_delta_mort_AVG,TDM_delta_mort_LOW,TDM_delta_mort_HIGH,total_delta_mort_AVG,total_delta_mort_LOW,total_delta_mort_HIGH
0,SSP1,2020.0,f,AFG,0.0,0.011760,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.594675,0.056406,1.025721
1,SSP1,2020.0,f,AFG,5.0,0.001139,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033347,0.003163,0.057519
2,SSP1,2020.0,f,AFG,10.0,0.000867,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019188,0.001820,0.033096
3,SSP1,2020.0,f,AFG,15.0,0.001353,0.006470,0.003094,0.011349,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012825,0.001216,0.022121
4,SSP1,2020.0,f,AFG,20.0,0.001860,0.005892,0.002868,0.009952,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013615,0.001291,0.023483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222383,SSP3,2050.0,m,ZWE,80.0,0.131022,0.041236,0.025676,0.061785,1.018463,...,0.005790,0.025175,0.017166,0.034006,0.014983,0.008152,0.022677,0.134066,0.062687,0.204309
222384,SSP3,2050.0,m,ZWE,85.0,0.192507,0.037578,0.024011,0.057540,1.018463,...,0.001896,0.014151,0.009649,0.019115,0.007526,0.004094,0.011390,0.074817,0.033583,0.115042
222385,SSP3,2050.0,m,ZWE,90.0,0.273660,0.025427,0.014721,0.037348,1.018463,...,0.000418,0.005447,0.003714,0.007357,0.002501,0.001361,0.003785,0.041221,0.013531,0.067200
222386,SSP3,2050.0,m,ZWE,95.0,0.363767,0.013584,0.007471,0.021601,1.018463,...,0.000043,0.001039,0.000709,0.001404,0.000362,0.000197,0.000548,0.015252,0.003138,0.026301


# premature deaths - interpolate the annual results (both adjusted and benchmark)

In [ ]:
columns_to_interpolate = [
    'Population','mx',
    'COP_delta_mort_AVG', 'COP_delta_mort_LOW', 'COP_delta_mort_HIGH',
    'IHD_delta_mort_AVG', 'IHD_delta_mort_LOW', 'IHD_delta_mort_HIGH',
    'LRI_delta_mort_AVG', 'LRI_delta_mort_LOW', 'LRI_delta_mort_HIGH',
    'LUN_delta_mort_AVG', 'LUN_delta_mort_LOW', 'LUN_delta_mort_HIGH',
    'STR_delta_mort_AVG', 'STR_delta_mort_LOW', 'STR_delta_mort_HIGH',
    'TDM_delta_mort_AVG', 'TDM_delta_mort_LOW', 'TDM_delta_mort_HIGH',
    'total_delta_mort_AVG', 'total_delta_mort_LOW', 'total_delta_mort_HIGH'
]

all_years = pd.DataFrame({'Year': np.arange(2020, 2051)})

def interpolate_group(group):

    constant_values = group[['SSPs','Scenario', 'sex', 'iso_code', 'age']].iloc[0]

    group = pd.merge(all_years, group, on='Year', how='left')

    group[columns_to_interpolate] = group[columns_to_interpolate].interpolate(method='linear')

    for col, val in constant_values.items():
        group[col] = val

    return group

columns_to_select = [
    'SSPs', 'Year', 'sex', 'iso_code', 'age', 'mx', 'Scenario', 'Population',
    'COP_delta_mort_AVG', 'COP_delta_mort_LOW', 'COP_delta_mort_HIGH',
    'IHD_delta_mort_AVG', 'IHD_delta_mort_LOW', 'IHD_delta_mort_HIGH',
    'LRI_delta_mort_AVG', 'LRI_delta_mort_LOW', 'LRI_delta_mort_HIGH',
    'LUN_delta_mort_AVG', 'LUN_delta_mort_LOW', 'LUN_delta_mort_HIGH',
    'STR_delta_mort_AVG', 'STR_delta_mort_LOW', 'STR_delta_mort_HIGH',
    'TDM_delta_mort_AVG', 'TDM_delta_mort_LOW', 'TDM_delta_mort_HIGH',
    'total_delta_mort_AVG', 'total_delta_mort_LOW', 'total_delta_mort_HIGH'
]

datasets = {
    
    'adjusted': adjusted_cumu_deaths,
    'benchmark': benchmark_cumu_deaths
}

for label, df in datasets.items():
    df_interpolated = (
        df[columns_to_select]
        .groupby(['SSPs', 'Scenario', 'sex', 'iso_code', 'age'], group_keys=False)
        .apply(interpolate_group)
    )

    df_interpolated.reset_index(drop=True, inplace=True)

    output_path = f'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/Health outcomes/Premature Deaths/{label}_cumu_deaths_fill.csv'
    df_interpolated.to_csv(output_path, index=False)

    globals()[f"{label}_cumu_deaths_fill"] = df_interpolated


/var/folders/w_/v7dhjtc519b5xkcdsf_td1480000gp/T/ipykernel_56656/2437090241.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(interpolate_group)
/var/folders/w_/v7dhjtc519b5xkcdsf_td1480000gp/T/ipykernel_56656/2437090241.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(interpolate_group)


# change in premature deaths - adjusted results compared to benchmark results

In [ ]:
merge_columns = ['SSPs', 'Year', 'sex', 'iso_code', 'age', 'mx', 'Population','Scenario']

diff_columns = [
    'COP_delta_mort_AVG', 'COP_delta_mort_LOW', 'COP_delta_mort_HIGH', 
    'IHD_delta_mort_AVG', 'IHD_delta_mort_LOW', 'IHD_delta_mort_HIGH', 
    'LRI_delta_mort_AVG', 'LRI_delta_mort_LOW', 'LRI_delta_mort_HIGH', 
    'LUN_delta_mort_AVG', 'LUN_delta_mort_LOW', 'LUN_delta_mort_HIGH', 
    'STR_delta_mort_AVG', 'STR_delta_mort_LOW', 'STR_delta_mort_HIGH', 
    'TDM_delta_mort_AVG', 'TDM_delta_mort_LOW', 'TDM_delta_mort_HIGH', 
    'total_delta_mort_AVG', 'total_delta_mort_LOW', 'total_delta_mort_HIGH'
]

change_cumu_deaths_fill = pd.merge(adjusted_cumu_deaths_fill[columns_to_select], benchmark_cumu_deaths_fill[columns_to_select],on=merge_columns, suffixes=('_adjusted', '_benchmark'),how='left')

for col in diff_columns:
    change_cumu_deaths_fill[f'{col}_diff'] = change_cumu_deaths_fill[f'{col}_adjusted'] - change_cumu_deaths_fill[f'{col}_benchmark']

scenario_rename_dict = {
  'SSP119':'SSP1-1.9','SSP126':'SSP1-2.6', 'SSP245':'SSP2-4.5','SSP370':'SSP3-7.0'}

change_cumu_deaths_fill['Scenario'] = change_cumu_deaths_fill['Scenario'].replace(scenario_rename_dict)

output_path = f'/Users/kediliukl/surfdrive/Documents/CML research/IIASA-YSSP/Liu et al code and dataset /Code&Data for results/Data/Health outcomes/Premature Deaths/change_cumu_deaths_fill.csv'

change_cumu_deaths_fill.to_csv(output_path, index=False)

change_cumu_deaths_fill

,SSPs,Year,sex,iso_code,age,mx,Scenario,Population,COP_delta_mort_AVG_adjusted,COP_delta_mort_LOW_adjusted,...,LUN_delta_mort_HIGH_diff,STR_delta_mort_AVG_diff,STR_delta_mort_LOW_diff,STR_delta_mort_HIGH_diff,TDM_delta_mort_AVG_diff,TDM_delta_mort_LOW_diff,TDM_delta_mort_HIGH_diff,total_delta_mort_AVG_diff,total_delta_mort_LOW_diff,total_delta_mort_HIGH_diff
0,SSP1,2020,f,AFG,0.0,0.011760,ssp119,3087.40,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,SSP1,2021,f,AFG,0.0,0.011089,ssp119,3101.88,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-0.002506,-0.000021,-0.007346
2,SSP1,2022,f,AFG,0.0,0.010417,ssp119,3116.36,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-0.005012,-0.000041,-0.014692
3,SSP1,2023,f,AFG,0.0,0.009746,ssp119,3130.84,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-0.007518,-0.000062,-0.022039
4,SSP1,2024,f,AFG,0.0,0.009074,ssp119,3145.32,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-0.010024,-0.000083,-0.029385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972211,SSP3,2046,m,ZWE,100.0,0.482033,ssp370,0.10,0.000053,0.000036,...,1.458089e-07,0.000003,0.000002,0.000005,0.000002,5.125166e-07,0.000003,0.000057,0.000006,0.000157
972212,SSP3,2047,m,ZWE,100.0,0.481921,ssp370,0.10,0.000054,0.000037,...,1.441616e-07,0.000003,0.000002,0.000005,0.000001,4.958434e-07,0.000003,0.000056,0.000006,0.000153
972213,SSP3,2048,m,ZWE,100.0,0.481810,ssp370,0.10,0.000055,0.000038,...,1.425144e-07,0.000003,0.000002,0.000004,0.000001,4.791702e-07,0.000003,0.000054,0.000006,0.000149
972214,SSP3,2049,m,ZWE,100.0,0.481698,ssp370,0.10,0.000057,0.000039,...,1.408672e-07,0.000003,0.000002,0.000004,0.000001,4.624970e-07,0.000003,0.000053,0.000005,0.000145
